# Ejercicio 4: Modelo Probabilístico

## Objetivo de la práctica
- Comprender los componentes del modelo vectorial mediante cálculos manuales y observación directa.
- Aplicar el modelo de espacio vectorial con TF-IDF para recuperar documentos relevantes.
- Comparar la recuperación con BM25 frente a TF-IDF.
- Analizar visualmente las diferencias entre los modelos.
- Evaluar si los rankings generados son consistentes con lo que considerarías documentos relevantes.

Creamos las funciona para obtener los datos de la matriz tf-idf

In [43]:
import numpy as np
import pandas as pd
import math
from collections import Counter, defaultdict
def calcular_tf(docs):
    tf_docs = []
    for doc in docs:
        total_palabras = len(doc)
        contador = Counter(doc)
        tf = {word: count / total_palabras for word, count in contador.items()}
        tf_docs.append(tf)
    return tf_docs

# 2. Calcular DF (Document Frequency) para el vocabulario
def calcular_df(docs, vocabulario):
    df = defaultdict(int)
    for palabra in vocabulario:
        df[palabra] = sum(1 for doc in docs if palabra in doc)
    return df

# 3. Crear matriz TF-IDF en DataFrame
def matriz_tfidf(docs, vocabulario):
    tf_docs = calcular_tf(docs)
    df = calcular_df(docs, vocabulario)
    N = len(docs)
    # Lista de diccionarios: cada dict es un documento con tf-idf por palabra
    tfidf_dicts = []
    for tf in tf_docs:
        tfidf = {}
        for palabra in vocabulario:
            tf_val = tf.get(palabra, 0)
            idf = math.log(N / df[palabra]) if df[palabra] > 0 else 0
            tfidf[palabra] = tf_val * idf
        tfidf_dicts.append(tfidf)
    # Convertir a DataFrame
    matriz = pd.DataFrame(tfidf_dicts)
    return matriz

## Parte 0: Carga del Corpus

In [7]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

## Parte 1: Cálculo de TF, DF, IDF y TF-IDF

### Actividad 
1. Utiliza el corpus cargado.
2. Construye la matriz de términos (TF), y calcula la frecuencia de documentos (DF)
3. Calcula TF-IDF utilizando sklearn.
4. Visualiza los valores en un DataFrame para analizar las diferencias entre los términos.

In [23]:
import re
def clean_text(texto):
    texto = texto.lower()                          # Minúsculas
    texto = re.sub(r'[^a-z\s]', '', texto)        # Solo letras y espacios
    texto = re.sub(r'\s+', ' ', texto).strip()    # Quitar espacios extras
    return texto

In [24]:
df = pd.DataFrame(newsgroupsdocs, columns=['doc'])

In [26]:
df_clean = df['doc'].apply(clean_text)

In [30]:
df_clean

0        i am sure some bashers of pens fans are pretty...
1        my brother is in the market for a highperforma...
2        finally you said what you dream about mediterr...
3        think its the scsi card doing the dma transfer...
4        i have an old jasmine drive which i cannot use...
                               ...                        
18841    dn from nyedacnsvaxuwecedu david nye dn a neur...
18842    not in isolated ground recepticles usually an ...
18843    i just installed a dx cpu in a clone motherboa...
18844    wouldnt this require a hypersphere in space po...
18845    after a tip from gary crum crumfcomccutahedu i...
Name: doc, Length: 18846, dtype: object

In [31]:
type(df_clean)

pandas.core.series.Series

In [32]:
tokenized_docs = df_clean.apply(lambda doc: doc.split())

In [44]:
tokenized_docs = df_clean.apply(lambda doc: doc.split()).tolist()
vocabulario = sorted(set(word for doc in tokenized_docs for word in doc))
vocabulario

['a',
 'aa',
 'aaa',
 'aaaaa',
 'aaaaaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg',
 'aaaaarrrrgh',
 'aaaall',
 'aaack',
 'aaaggghhh',
 'aaah',
 'aaahh',
 'aaahhhh',
 'aaai',
 'aaaimit',
 'aaalwhtpao',
 'aaamajors',
 'aaareadme',
 'aablqyabj',
 'aabxhaafqmbmiqkxgk',
 'aac',
 'aacc',
 'aacs',
 'aacvmjrfxidwo',
 'aadams',
 'aadnsi',
 'aaer',
 'aaffff',
 'aafffff',
 'aafreenetcarletonca',
 'aagain',
 'aagrendalcorpsuncom',
 'aah',
 'aaigcapcorg',
 'aalmchgurmeyaridiylpehpaifhnfmqqlchvcduajjebndih',
 'aalocutuscscoloradoedu',
 'aalong',
 'aalternate',
 'aam',
 'aamazing',
 'aamir',
 'aammmaaaazzzzzziinnnnggggg',
 'aamothrasyredu',
 'aamydualuucp',
 'aan',
 'aanbieden',
 'aand',
 'aandpyq',
 'aanerud',
 'aangeboden',
 'aangegeven',
 'aangezien',
 'aantal',
 'aantalsnijpunten',
 'aao',
 'aap',
 'aaph',
 'aapizzaboxdemoncouk',
 'aapjj',
 'aaplay',
 'aaplayexe',
 'aapp',
 'aargh',
 'aarguing',
 'aarhus',
 'aarm',
 'aarnet',
 'aaron',
 'aaroncathe

In [ ]:
    matriz_tfidf_final = matriz_tfidf(tokenized_docs, vocabulario)

In [41]:
print(list(tfidf_docs[0].items())[:10])

[('i', 0.016358845863466618), ('am', 0.04417256650967318), ('sure', 0.018745922395767656), ('some', 0.01075215538342004), ('bashers', 0.06010670114111388), ('of', 0.0226266110512649), ('pens', 0.20369985736224036), ('fans', 0.03407230829337863), ('are', 0.02049421216303317), ('pretty', 0.023561976522875065)]


In [42]:
V = len(vocabulario)
matriz_tfidf = np.zeros((N, V))

MemoryError: Unable to allocate 17.1 GiB for an array with shape (18846, 121462) and data type float64

Creamos una funciona para obtener el vocabulario

In [12]:
vocab = get_vocab(df_clean)

Usamos la funcion para obtener la matriz TF-IDF

In [43]:
vector_TF(df_clean[5], vocab)

array([0, 0, 0, ..., 0, 0, 0], shape=(156023,))

In [ ]:
matrizTF = matrix_TF(df_clean, vocab)

In [45]:
matrizTF

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(20, 156023))

## Parte 2: Ranking de documentos usando TF-IDF

### Actividad 

1. Dada una consulta, construye el vector de consulta
2. Calcula la similitud coseno entre la consulta y cada documento usando los vectores TF-IDF
3. Genera un ranking de los documentos ordenados por relevancia.
4. Muestra los resultados en una tabla.

## Parte 3: Ranking con BM25

### Actividad 

1. Implementa un sistema de recuperación usando el modelo BM25.
2. Usa la misma consulta del ejercicio anterior.
3. Calcula el score BM25 para cada documento y genera un ranking.
4. Compara manualmente con el ranking de TF-IDF.

## Parte 4: Comparación visual entre TF-IDF y BM25

### Actividad 

1. Utiliza un gráfico de barras para visualizar los scores obtenidos por cada documento según TF-IDF y BM25.
2. Compara los rankings visualmente.
3. Identifica: ¿Qué documentos obtienen scores más altos en un modelo que en otro?
4. Sugiere: ¿A qué se podría deber esta diferencia?

## Parte 5: Evaluación con consulta relevante

### Actividad 

1. Elige una consulta y define qué documentos del corpus deberían considerarse relevantes.
2. Evalúa Precision@3 o MAP para los rankings generados con TF-IDF y BM25.
3. Responde: ¿Cuál modelo da mejores resultados respecto a tu criterio de relevancia?